In [3]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "4"
os.environ["NUM_THREADS"] = "4"
os.environ["OMP_NUM_THREADS"] = "4"
from keras.layers import Input, Dense, Flatten, Reshape
from sklearn.feature_extraction import image as sk_image
from concurrent.futures import ProcessPoolExecutor
import cartopy.feature as cfeature
from keras.models import Model
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sklearn.cluster import AgglomerativeClustering
from scipy.signal import convolve2d 
from scipy import ndimage
from sklearn.cluster import KMeans
import numpy as np
import tensorflow as tf
from tensorflow import keras    
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from sklearn.cluster import DBSCAN
from pyhdf.SD import SD, SDC
import matplotlib as mpl
#tf.config.threading.set_inter_op_parallelism_threads(1)
from extract_training_data import *
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from pyhdf.error import HDF4Error
from functions import *
from tensorflow.keras.models import load_model
from sklearn.cluster import KMeans, MiniBatchKMeans
import joblib
import plot_functions
import importlib 
importlib.reload(plot_functions)
from plot_functions import *
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
# Visualize the result

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,"  , len(logical_gpus), "Logical GPUs")
        tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

import socket
hostname = socket.gethostname()
if "nird" in hostname:
    tf.config.threading.set_inter_op_parallelism_threads(8)
    data_loc = "/nird/projects/NS9600K/fslippe/data/"
    folder = "/nird/projects/NS9600K/data/modis/cao/MOD02/2020/ /nird/projects/NS9600K/data/modis/cao/MOD02/2021/ /nird/projects/NS9600K/data/modis/cao/MOD02/2023/"
if "mimi" in hostname:
    data_loc = "/uio/hume/student-u37/fslippe/data/"
    folder = "/scratch/fslippe/modis/MOD02/daytime_1km/ /scratch/fslippe/modis/MOD02/boundary_1km/ /scratch/fslippe/modis/MOD02/night_1km/ /scratch/fslippe/modis/MOD02/may-nov_2021/ /scratch/fslippe/modis/MOD02/cao_test_data/"


bands=[29]
from autoencoder import SobelFilterLayer, SimpleAutoencoder
print(len(bands))

#encoder = load_model("/uio/hume/student-u37/fslippe/data/models/winter_2020_21_band(6,20,29)_encoder")
max_vals = np.load("/uio/hume/student-u37/fslippe/data/models/patch_size128/max_val_dnb_l95_z50_ps128_band29_2018-2023.npy")
min_vals = np.load("/uio/hume/student-u37/fslippe/data/models/patch_size128/min_val_dnb_l95_z50_ps128_band29_2018-2023.npy")


1 Physical GPUs, 1 Logical GPUs
1


In [5]:
### EXTRACT CAO AND NOn CAO CASES
import importlib
import extract_training_data
importlib.reload(extract_training_data)
from extract_training_data import *
patch_size = 128
year = 2023
start = f"{year}0101"
end = f"{year}0430"
dates = generate_date_list(start, end)

start = f"{year}1001"
end = f"{year}1231"
dates.extend(generate_date_list(start, end))


folder = "/scratch/fslippe/modis/MOD02/2019/ /scratch/fslippe/modis/MOD02/2020/ /scratch/fslippe/modis/MOD02/2021/ /scratch/fslippe/modis/MOD02/2022/ /scratch/fslippe/modis/MOD02/2023/"
x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao = extract_1km_data(folder,
                                                         bands=bands,
                                                         date_list=dates,
                                                         return_lon_lat=True,
                                                         data_loc=data_loc,
                                                         data_type="npy",
                                                         combine_pics=True)

x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao = zip(*[(xi, date, mask, lon_lat, mod_min) for xi, date, mask, lon_lat, mod_min in zip(x_cao, dates_cao, masks_cao, lon_lats_cao, mod_min_cao) if (xi.shape[0] > patch_size) and (xi.shape[1] > patch_size)])
x_cao = list(x_cao)
dates_cao = list(dates_cao)
len(x_cao)


total cores: 256
['/scratch/fslippe/modis/MOD02/2019/', '/scratch/fslippe/modis/MOD02/2020/', '/scratch/fslippe/modis/MOD02/2021/', '/scratch/fslippe/modis/MOD02/2022/', '/scratch/fslippe/modis/MOD02/2023/']
['2019001', '2019002', '2019003', '2019004', '2019005', '2019006', '2019007', '2019008', '2019009', '2019010', '2019011', '2019012', '2019013', '2019014', '2019015', '2019016', '2019017', '2019018', '2019019', '2019020', '2019021', '2019022', '2019023', '2019024', '2019025', '2019026', '2019027', '2019028', '2019029', '2019030', '2019031', '2019032', '2019033', '2019034', '2019035', '2019036', '2019037', '2019038', '2019039', '2019040', '2019041', '2019042', '2019043', '2019044', '2019045', '2019046', '2019047', '2019048', '2019049', '2019050', '2019051', '2019052', '2019053', '2019054', '2019055', '2019056', '2019057', '2019058', '2019059', '2019060', '2019061', '2019062', '2019063', '2019064', '2019065', '2019066', '2019067', '2019068', '2019069', '2019070', '2019071', '2019072',

100%|██████████| 196/196 [03:26<00:00,  1.05s/it]


2348
2348


2008

In [6]:
autoencoder_predict = SimpleAutoencoder(len(bands), patch_size, patch_size)
patches, all_lon_patches, all_lat_patches, starts, ends, shapes, n_patches_tot, indices = generate_patches([x[:,:,0] for x in x_cao],
                                                                                                    masks_cao,
                                                                                                    lon_lats_cao,
                                                                                                    max_vals,
                                                                                                    min_vals,
                                                                                                    autoencoder_predict,
                                                                                                    strides=[1, patch_size, patch_size,1])

                                                                                                    

In [7]:
filter = 128

if filter == 32:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter32/encoder_dnb_l95_z50_ps128_f32_1e3_201812-202312.h5")
elif filter == 64:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter64/encoder_dnb_l95_z50_ps128_f64_1e3_201812-202312_epoch_500.h5")
elif filter == 128:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter128/encoder_dnb_l95_z50_ps128_f128_1e3_201812-202312.h5")


n_chunks = 40
n_patches = len(patches)
chunk_size = n_patches // n_chunks

patch_chunks = [patches[i * chunk_size : (i + 1) * chunk_size] for i in range(n_chunks)]

# Handle the remaining patches for the case where n_patches is not exactly divisible by n_chunks
if n_patches % n_chunks != 0:
    patch_chunks.append(patches[n_chunks * chunk_size:])

encoded_patch_chunks = [encoder.predict(chunk) for chunk in patch_chunks]
encoded_patches = np.concatenate(encoded_patch_chunks)

encoded_patches_flat = encoded_patches.reshape(encoded_patches.shape[0], -1)
gc.collect()

2024-01-27 21:05:59.869465: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


30865

In [8]:
import plot_functions
importlib.reload(plot_functions)
from plot_functions import * 
thresholds = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

n_Ks = [10, 11, 12, 13, 14, 15, 16]
for n_K in n_Ks:
    cluster =  joblib.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}.pkl")
    label_1 = int(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}_opencell_label.npy"))
    label_2 = int(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}_closedcell_label.npy"))
    print(label_1, label_2)

    labels = cluster.predict(encoded_patches_flat)
    folder = f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{filter}/clustering/cao_date_time_lists/n_K_{n_K}/"
    if not os.path.exists(folder):
        os.makedirs(folder)

    for threshold in thresholds:
        dates, times = save_img_with_labels(x_cao,
                            lon_lats_cao,
                            n_patches_tot,
                            indices,
                            labels,
                            starts,
                            ends,  
                            shapes,
                            dates_cao,
                            mod_min_cao,
                            plot=False,
                            less_than=False,
                            max_pics = 10000,
                            desired_label=[label_1,label_2],
                            size_threshold=threshold,
                            patch_size=patch_size,
                            global_max=n_K)
        print(len(dates))
        time_dict = {"dates": dates, "times": times}
        np.save(folder + f"times_patch_size{patch_size}_filter{filter}_nK{n_K}_thr{threshold}_{year}", time_dict)

        



4 7
308
166
104
71
51
36
30
16
10
7
5 0
351
195
114
76
50
36
23
14
9
6
8 10
218
121
72
48
36
29
18
10
8
7
5 4
232
120
74
45
36
27
18
7
6
4
9 13
189
109
66
47
35
27
15
7
6
3
9 6
134
58
32
17
12
5
1
1
1
1
12 2
168
83
51
33
27
14
9
7
6
2


In [9]:
filter = 64

if filter == 32:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter32/encoder_dnb_l95_z50_ps128_f32_1e3_201812-202312.h5")
elif filter == 64:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter64/encoder_dnb_l95_z50_ps128_f64_1e3_201812-202312_epoch_500.h5")
elif filter == 128:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter128/encoder_dnb_l95_z50_ps128_f128_1e3_201812-202312.h5")


n_chunks = 40
n_patches = len(patches)
chunk_size = n_patches // n_chunks

patch_chunks = [patches[i * chunk_size : (i + 1) * chunk_size] for i in range(n_chunks)]

# Handle the remaining patches for the case where n_patches is not exactly divisible by n_chunks
if n_patches % n_chunks != 0:
    patch_chunks.append(patches[n_chunks * chunk_size:])

encoded_patch_chunks = [encoder.predict(chunk) for chunk in patch_chunks]
encoded_patches = np.concatenate(encoded_patch_chunks)

encoded_patches_flat = encoded_patches.reshape(encoded_patches.shape[0], -1)
gc.collect()

import plot_functions
importlib.reload(plot_functions)
from plot_functions import * 
thresholds = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

n_Ks = [10, 11, 12, 13, 14, 15, 16]
for n_K in n_Ks:
    cluster =  joblib.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}.pkl")
    label_1 = int(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}_opencell_label.npy"))
    label_2 = int(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}_closedcell_label.npy"))
    print(label_1, label_2)

    labels = cluster.predict(encoded_patches_flat)
    folder = f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{filter}/clustering/cao_date_time_lists/n_K_{n_K}/"
    if not os.path.exists(folder):
        os.makedirs(folder)

    for threshold in thresholds:
        dates, times = save_img_with_labels(x_cao,
                            lon_lats_cao,
                            n_patches_tot,
                            indices,
                            labels,
                            starts,
                            ends,  
                            shapes,
                            dates_cao,
                            mod_min_cao,
                            plot=False,
                            less_than=False,
                            max_pics = 10000,
                            desired_label=[label_1,label_2],
                            size_threshold=threshold,
                            patch_size=patch_size,
                            global_max=n_K)
        print(len(dates))
        time_dict = {"dates": dates, "times": times}
        np.save(folder + f"times_patch_size{patch_size}_filter{filter}_nK{n_K}_thr{threshold}_{year}", time_dict)

        



4 7
1064
480
236
134
81
57
38
27
18
11
3 0
1063
480
236
135
80
57
38
26
19
12
1 3
306
155
93
57
37
24
14
11
7
6
0 9
260
120
67
38
28
18
9
7
3
3
5 10
285
160
98
58
40
29
17
11
7
4
11 3
282
159
97
59
40
30
18
11
7
4
13 4
281
160
96
59
40
30
17
11
7
4


In [10]:
filter = 32

if filter == 32:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter32/encoder_dnb_l95_z50_ps128_f32_1e3_201812-202312.h5")
elif filter == 64:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter64/encoder_dnb_l95_z50_ps128_f64_1e3_201812-202312_epoch_500.h5")
elif filter == 128:
    encoder = load_model("/uio/hume/student-u37/fslippe/data/models/patch_size128/filter128/encoder_dnb_l95_z50_ps128_f128_1e3_201812-202312.h5")


n_chunks = 40
n_patches = len(patches)
chunk_size = n_patches // n_chunks

patch_chunks = [patches[i * chunk_size : (i + 1) * chunk_size] for i in range(n_chunks)]

# Handle the remaining patches for the case where n_patches is not exactly divisible by n_chunks
if n_patches % n_chunks != 0:
    patch_chunks.append(patches[n_chunks * chunk_size:])

encoded_patch_chunks = [encoder.predict(chunk) for chunk in patch_chunks]
encoded_patches = np.concatenate(encoded_patch_chunks)

encoded_patches_flat = encoded_patches.reshape(encoded_patches.shape[0], -1)
gc.collect()

import plot_functions
importlib.reload(plot_functions)
from plot_functions import * 
thresholds = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

n_Ks = [10, 11, 12, 13, 14, 15, 16]
for n_K in n_Ks:
    cluster =  joblib.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}.pkl")
    label_1 = int(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}_opencell_label.npy"))
    label_2 = int(np.load(f"/uio/hume/student-u37/fslippe/data/models/patch_size128/filter{filter}/clustering/cluster_dnb_l95_z50_ps128_band29_filter{filter}_K{n_K}_closedcell_label.npy"))
    print(label_1, label_2)

    labels = cluster.predict(encoded_patches_flat)
    folder = f"/uio/hume/student-u37/fslippe/data/models/patch_size{patch_size}/filter{filter}/clustering/cao_date_time_lists/n_K_{n_K}/"
    if not os.path.exists(folder):
        os.makedirs(folder)

    for threshold in thresholds:
        dates, times = save_img_with_labels(x_cao,
                            lon_lats_cao,
                            n_patches_tot,
                            indices,
                            labels,
                            starts,
                            ends,  
                            shapes,
                            dates_cao,
                            mod_min_cao,
                            plot=False,
                            less_than=False,
                            max_pics = 10000,
                            desired_label=[label_1,label_2],
                            size_threshold=threshold,
                            patch_size=patch_size,
                            global_max=n_K)
        print(len(dates))
        time_dict = {"dates": dates, "times": times}
        np.save(folder + f"times_patch_size{patch_size}_filter{filter}_nK{n_K}_thr{threshold}_{year}", time_dict)

        



6 5
1474
1025
662
429
285
170
107
49
27
15
3 9
1093
492
227
105
43
18
9
3
1
0
7 9
1302
774
430
245
146
78
47
25
11
4
2 10
770
261
112
46
14
5
1
0
0
0
4 2
1489
1078
676
436
285
191
114
73
47
26
1 8
1397
902
539
355
220
135
83
50
30
11
10 11
1263
720
418
250
151
89
49
28
13
6
